In [1]:
import os
import shutil

from typing import List
from datetime import datetime
from configparser import ConfigParser

from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
import time


In [ ]:
companies = ['ALRS', 'AFLT', 'VTBR', 'GAZP', 'GMKN', 'LSRG', 'IRAO', 'LKOH', 'MAGN', 'MOEX', 'CBOM', 'MTSS',
             'ROSN', 'NLMK', 'NVTK', 'PIKK', 'PLZL', 'RTKM', 'RUAL', 'HYDR', 'SBER', 'CHMF', 'AFKS', 'SNGS',
             'TATN', 'TRNFP', 'PHOR', 'FEES']
start_year = 2023
end_year = 2024
save_root = D:/Jora/Article 2 Data/Time Series

In [ ]:
month_dct = {0: 'январь', 1: 'февраль', 2: 'март', 3: 'апрель', 4: 'май', 5: 'июнь', 6: 'июль', 7: 'август', 8: 'сентябрь', 9: 'октябрь',
             10: 'ноябрь', 11: 'декабрь'}

def scroll_calendar(driver: webdriver.Chrome, action: ActionChains, key_month: int, key_year: int, first: bool):
    while True:
        month = driver.find_element(By.XPATH, '//p[@class="ui-kit-Body14-ui-kit-18ajdke cApxOD ui-kit-Month-ui-kit-b7tvtb iSnnyt ui-calendar__month weightMedium"]').text.lower()
        year = int(driver.find_element(By.XPATH, '//input[@class="ui-kit-YearInput-ui-kit-x3evgy jKzXzQ ui-calendar__year"]').get_attribute('value'))
        if (month == month_dct[key_month] and key_year == year) or (datetime.now().month - 1 <= key_month and year == key_year):
            break
        button = driver.find_element(By.XPATH, '//button[@class="ui-kit-Root-ui-kit-1dybyb0 hOoKPm ui-calendar__button-left"]')
        action.click(button)
        action.perform()
        time.sleep(2)

    if first:
        start_date_element = driver.find_elements(By.XPATH, '//button[@class="ui-kit-Root-ui-kit-1dybyb0 hOoKPm ui-kit-Root-ui-kit-1nxmom8 gSfMwq"]')[0]
    else:
        start_date_element = driver.find_elements(By.XPATH, '//button[@class="ui-kit-Root-ui-kit-1dybyb0 hOoKPm ui-kit-Root-ui-kit-1nxmom8 gSfMwq"]')[-1]

    action.click(start_date_element)
    action.perform()

def parse_finam(link: str, start_year: int, end_year: int, save_root: str='D:/Жора/Article 2 Data/Time Series/'):
    driver = webdriver.Chrome()
    driver.get(link)
    action = ActionChains(driver)

    # Переодичность
    ticks = driver.find_element(by=By.XPATH, value='//p[@class="ui-kit-Body14-ui-kit-18ajdke cApxOD ui-kit-DisplayValue-ui-kit-w5feoh ikFxEs ui-select-field__value"]')
    action.click(ticks)
    action.perform()
    time.sleep(0.5)
    min_1 = driver.find_element(by=By.XPATH, value='//li[contains(text(), "1 мин.")]')
    action.click(min_1)
    action.perform()

    # Тип файла
    dtype = driver.find_element(by=By.XPATH, value='//p[contains(text(), "Тип файла")]')
    action.click(dtype)
    action.perform()
    time.sleep(0.5)
    csv = driver.find_element(by=By.XPATH, value='//li[contains(text(), ".csv")]')
    action.click(csv)
    action.perform()

    for year in range(end_year, start_year-1, -1):
        for month in [0, 3, 6, 9][::-1]:

            # Дата начала парсинга
            from_window = driver.find_element(by=By.NAME, value='from')
            action.click(from_window)
            action.perform()

            scroll_calendar(driver, action, month, year, True)

            # Дата конца парсинга
            to_window = driver.find_element(by=By.NAME, value='to')
            action.click(to_window)
            action.perform()

            scroll_calendar(driver, action, month+2, year, False)

            get_file = driver.find_element(by=By.XPATH, value='//button[@class="ui-kit-BaseButton-ui-kit-ekw2d7 ui-kit-PrimaryButton-ui-kit-1j75dgi bgmPVI dzTiIc sizeMedium colorPrimary"]')
            action.click(get_file)
            action.perform()
            time.sleep(5)
    time.sleep(5)
    driver.quit()
    

def get_all_quotes(companies: List[str], start_year: int=2016, end_year: int=2024):
    for company in companies:
        try:
            link = f'https://www.finam.ru/quote/moex/{company.lower()}/export/'
            parse_finam(link, start_year, end_year)
            print(f'{company} parsed')
        except KeyboardInterrupt:
            raise
        except Exception as ex:
            print(ex)

def move_all_to_folder(companies, root='D:/Jora/Article 2 Data/Time Series'):
    files = [el for el in os.listdir('C:/Users/ап/Downloads') if el.startswith(tuple(companies))]
    for file in files:
        shutil.move(f'C:/Users/ап/Downloads/{file}', f'{root}/{file}')

if __name__ == '__main__':
    config = ConfigParser()
    config.read('configs/finam.ini')
    get_all_quotes(eval(config['GENERAL']['companies']), eval(config['GENERAL']['start_year']), eval(config['GENERAL']['end_year']))
    move_all_to_folder(eval(config['GENERAL']['companies']), config['GENERAL']['save_root'])

**Код GPT без конфига:**

In [4]:
import os
import shutil
from typing import List
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

month_dct = {0: 'январь', 1: 'февраль', 2: 'март', 3: 'апрель', 4: 'май', 5: 'июнь', 6: 'июль', 7: 'август', 8: 'сентябрь', 9: 'октябрь',
             10: 'ноябрь', 11: 'декабрь'}

def scroll_calendar(driver: webdriver.Chrome, action: ActionChains, key_month: int, key_year: int, first: bool):
    while True:
        month = driver.find_element(By.XPATH, '//p[@class="ui-kit-Body14-ui-kit-18ajdke cApxOD ui-kit-Month-ui-kit-b7tvtb iSnnyt ui-calendar__month weightMedium"]').text.lower()
        year = int(driver.find_element(By.XPATH, '//input[@class="ui-kit-YearInput-ui-kit-x3evgy jKzXzQ ui-calendar__year"]').get_attribute('value'))
        if (month == month_dct[key_month] and key_year == year) or (datetime.now().month - 1 <= key_month and year == key_year):
            break
        button = driver.find_element(By.XPATH, '//button[@class="ui-kit-Root-ui-kit-1dybyb0 hOoKPm ui-calendar__button-left"]')
        action.click(button)
        action.perform()
        time.sleep(2)

    if first:
        start_date_element = driver.find_elements(By.XPATH, '//button[@class="ui-kit-Root-ui-kit-1dybyb0 hOoKPm ui-kit-Root-ui-kit-1nxmom8 gSfMwq"]')[0]
    else:
        start_date_element = driver.find_elements(By.XPATH, '//button[@class="ui-kit-Root-ui-kit-1dybyb0 hOoKPm ui-kit-Root-ui-kit-1nxmom8 gSfMwq"]')[-1]

    action.click(start_date_element)
    action.perform()
    
def parse_finam(link: str, start_year: int, end_year: int, save_root: str = 'Stock_data'):
    driver = webdriver.Chrome()
    driver.get(link)
    action = ActionChains(driver)

    try:
        # Переодичность
        ticks = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, '//p[@class="ui-kit-Body14-ui-kit-18ajdke cApxOD ui-kit-DisplayValue-ui-kit-w5feoh ikFxEs ui-select-field__value"]'))
        )
        action.click(ticks).perform()
        min_1 = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, '//li[contains(text(), "1 мин.")]'))
        )
        action.click(min_1).perform()

        # Тип файла
        dtype = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, '//p[contains(text(), "Тип файла")]'))
        )
        action.click(dtype).perform()
        csv = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, '//li[contains(text(), ".csv")]'))
        )
        action.click(csv).perform()

        for year in range(end_year, start_year - 1, -1):
            for month in [0, 3, 6, 9][::-1]:
                # Дата начала парсинга
                from_window = WebDriverWait(driver, 10).until(
                    EC.element_to_be_clickable((By.NAME, 'from'))
                )
                action.click(from_window).perform()
                scroll_calendar(driver, action, month, year, True)

                # Дата конца парсинга
                to_window = WebDriverWait(driver, 10).until(
                    EC.element_to_be_clickable((By.NAME, 'to'))
                )
                action.click(to_window).perform()
                scroll_calendar(driver, action, month + 2, year, False)

                # Кнопка загрузки файла
                get_file = WebDriverWait(driver, 10).until(
                    EC.element_to_be_clickable((By.XPATH, '//button[@class="ui-kit-BaseButton-ui-kit-ekw2d7 ui-kit-PrimaryButton-ui-kit-1j75dgi bgmPVI dzTiIc sizeMedium colorPrimary"]'))
                )
                action.click(get_file).perform()

                # Ожидание загрузки файла
                time.sleep(5)  # Возможно стоит заменить на более надежный способ проверки загрузки

    except Exception as e:
        print(f"Произошла ошибка: {e}")

    finally:
        driver.quit()

def get_all_quotes(companies: List[str], start_year: int = 2019, end_year: int = 2025):
    for company in companies:
        try:
            link = f'https://www.finam.ru/quote/moex/{company.lower()}/export/'
            parse_finam(link, start_year, end_year)
            print(f'{company} parsed')
        except KeyboardInterrupt:
            raise
        except Exception as ex:
            print(ex)

def move_all_to_folder(companies: List[str], root: str = 'Stock_data'):
    files = [el for el in os.listdir('/Users/irina/Downloads') if el.startswith(tuple(companies))]
    for file in files:
        shutil.move(f'/Users/irina/Downloads/{file}', f'{root}/{file}')

if __name__ == '__main__':
    #companies = ['ALRS', 'AFLT', 'VTBR', 'GAZP', 'GMKN', 'LSRG', 'IRAO', 'LKOH', 'MAGN', 'MOEX', 
     #            'CBOM', 'MTSS', 'ROSN', 'NLMK', 'NVTK', 'PIKK', 'PLZL', 'RTKM', 'RUAL', 
      #           'HYDR', 'SBER', 'CHMF', 'AFKS', 'SNGS', 'TATN', 'TRNFP', 'PHOR', 'FEES']
    companies = ['SBER']
    start_year = 2019
    end_year = 2025
    save_root = 'Stock_data'

    get_all_quotes(companies, start_year, end_year)
    

Произошла ошибка: Message: 
Stacktrace:
0   chromedriver                        0x0000000102652dec cxxbridge1$str$ptr + 2817040
1   chromedriver                        0x000000010264b088 cxxbridge1$str$ptr + 2784940
2   chromedriver                        0x00000001021928d8 cxxbridge1$string$len + 93028
3   chromedriver                        0x00000001021d96a0 cxxbridge1$string$len + 383276
4   chromedriver                        0x000000010221a7b8 cxxbridge1$string$len + 649796
5   chromedriver                        0x00000001021cda80 cxxbridge1$string$len + 335116
6   chromedriver                        0x0000000102617c74 cxxbridge1$str$ptr + 2575000
7   chromedriver                        0x000000010261af40 cxxbridge1$str$ptr + 2588004
8   chromedriver                        0x00000001025f79fc cxxbridge1$str$ptr + 2443296
9   chromedriver                        0x000000010261b7bc cxxbridge1$str$ptr + 2590176
10  chromedriver                        0x00000001025e8af0 cxxbridge1$str

In [25]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--verbose')
driver = webdriver.Chrome(options=chrome_options)

In [3]:
companies = {'ALRS': 'alrosa-ao', 'AFLT': 'aeroflot', 'VTBR': 'vtb', 'GAZP' : 'gazprom', 'GMKN': 'nornickel-gmk', 'LSRG': 'lsr',
             'DSKY': 'detsky-mir', 'IRAO': 'inter-rao-ao', 'LKOH': 'lukoil', 'MAGN': 'mmk', 'MOEX': 'moscowexchange', 'CBOM': 'oao-mkb-ao',
             'MTSS': 'mts', 'VKCO': 'gdr-vk-company-limited-ord-shs_vkco-tqbr', 'ROSN': 'rosneft', 'NLMK': 'nlmk-ao', 'NVTK': 'novatek',
             'PIKK': 'pik', 'PLZL': 'polus-zoloto', 'RTKM': 'rostelecom', 'RUAL': 'united-company-rusal-plc', 'HYDR': 'rusgidro',
             'SBER': 'sberbank', 'CHMF': 'severstal-ao', 'AFKS': 'afk-sistema', 'SNGS': 'surgut', 'TATN': 'tatneft-3', 'TCSG': 'gdr-tcs-group-holding-ord-shs_tcsg',
             'TRNFP': 'transneft-pref', 'POGR': 'petropavlovsk-plc_pogr', 'PHOR': 'phosagro-ao', 'FEES': 'fsk-ees', 'HHRU': 'adr-headhunter-group-ord-shs_hhru',
             'YNDX': 'pllc-yandex-n-v'}

In [3]:
# link = 'https://www.finam.ru/profile/moex-akcii/pllc-yandex-n-v/export/'


Message: ; For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception

Message: Could not create a session: The Safari instance is already paired with another WebDriver session.

Message: Could not create a session: The Safari instance is already paired with another WebDriver session.

Message: Could not create a session: The Safari instance is already paired with another WebDriver session.

Message: Could not create a session: The Safari instance is already paired with another WebDriver session.

Message: Could not create a session: The Safari instance is already paired with another WebDriver session.



NameError: name 'safaridriver' is not defined